In [ ]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.5 MB 483 kB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

import re

/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


In [ ]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631.0,1.0,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631.0,1.0,0.0,0.0


In [ ]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [ ]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    proba = items_weights['weight'].to_list()

    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=proba)
    
    
    return recs.tolist()    

In [ ]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights.columns = ['item_id', 'weight']

items_weights = items_weights[items_weights['weight'] > 1]
items_weights['weight'] = np.log(items_weights['weight'])

items_weights['weight'] = items_weights['weight'] / items_weights['weight'].sum()

print(items_weights)

      item_id    weight
0       33150  0.000226
1       34198  0.000129
2       39592  0.000226
3       40885  0.000134
4       50905  0.000094
...       ...       ...
6069  9882002  0.000323
6070  9884041  0.000226
6071  9884379  0.000478
6072  9884933  0.000226
6073  9885390  0.000380

[5435 rows x 2 columns]
CPU times: user 13.4 ms, sys: 1.85 ms, total: 15.3 ms
Wall time: 22.9 ms


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [ ]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [ ]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [ ]:
print(type(result['actual'][0]))
print(result['actual'][0])

<class 'str'>
[  821867   834484   856942   865456   889248   907957   914190   943316
   951954   954486   958046   962568   969231   971585   979707   986947
   990656   995242  1004906  1005186  1042083  1050310  1060819  1062002
  1064441  1069733  1074612  1082185  1131115  1132771  6534544 13876341
 15971874 17178953   883616   917704   931860   961554  1002032  1031190
  8090541  8293439  9297615  9527329 15926712  1049998   861272   869465
   877373   908213   933913   940947   945809   959316   978974  1031697
  1041796  1048918  1081189  1101422  1115576  1122428  1132231  1132814
  5577022  8091601  9296986  9677939 10356149 13417048 15741823 15830875]


In [ ]:
# файл с предсказаниями сохранил предсказания как строки
# нужно перевести обратно в список
for a in result.columns[1:]:
    result.loc[:, a]=  result[a].map(lambda x: re.findall(r'[0-9][0-9]*', x[1:-1])).apply(lambda x: list(map(int, x)))

In [ ]:
print(type(result['actual'][0]))
print(result['actual'][1])

<class 'list'>
[835476, 851057, 872021, 878302, 879948, 909638, 913202, 920626, 958154, 994891, 1053690, 1083328, 1096727, 6463658, 7167218, 7167249, 9526563, 9526886, 13842214]


In [ ]:
result.apply(lambda x: precision_at_k(x['own_purchases'], x['actual'],  5), axis=1).mean()

0.17998694090760692

In [ ]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")


1.0:actual
0.0006:random_recommendation
0.1552:popular_recommendation
0.0336:itemitem
0.0353:cosine
0.0361:tfidf
0.18:own_purchases


Лучшее качество показал алгоритм popular_recommendation

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code